In [7]:
import os

In [8]:
# change these to try this notebook out
BUCKET = 'gs://edml/data/taxi-trips'

In [9]:
os.environ['BUCKET'] = BUCKET

## Local training / Python

In [10]:
%%bash
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
echo $PYTHONPATH

:/home/jupyter/event-driven-ml/edml-trainer


In [11]:
%%bash
echo "bucket=${BUCKET}"
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
python3 -m trainer.task \
  --bucket=${BUCKET} \
  --output-dir=model_trained_test \
  --job-dir=./tmp \
  --pattern="*" --train-examples=500 --eval-steps=1

bucket=gs://edml/data/taxi-trips
Will train for 976.5625 steps using batch_size=512
Will use DNN size of [128, 32, 4]


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.datalab.ml import TensorBoard

In [25]:
TensorBoard().start('./model_trained_test')
TensorBoard().list()

TensorBoard was started successfully with pid 3973. Click here to access it.

,logdir,pid,port
0,./model_trained_test,3973,42081


In [26]:
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 3973


In [11]:
# %%bash
# export MODEL_DIR=tmp_tensorboard
# tensorboard --logdir=$MODEL_DIR --port=6006

Process is terminated.


## Local prediction

In [12]:
%%writefile inputs.json
{"uuid": "b1", "dayofweek": 6, "hourofday": 7, "pickup_borough": "Queens", "dropoff_borough": "Bronx"}
{"uuid": "g1", "dayofweek": 3, "hourofday": 23, "pickup_borough": "Manhattan", "dropoff_borough": "Queens"}

Writing inputs.json


In [13]:
%%bash
MODEL_LOCATION=$(ls -d $(pwd)/model_trained_test/export/exporter/* | tail -1)
echo $MODEL_LOCATION
gcloud ai-platform local predict --model-dir=$MODEL_LOCATION --json-instances=inputs.json

/home/jupyter/event-driven-ml/edml-notebooks/gbi/model_trained_test/export/exporter/1573144860
PREDICTIONS
[7.751214981079102]
[8.675673484802246]


If the signature defined in the model is not serving_default then you must specify it via --signature-name flag, otherwise the command may fail.






2019-11-07 16:41:46.827335: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-07 16:41:46.836000: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-11-07 16:41:46.836275: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ed8b41f9f0 executing computations on platform Host. Devices:
2019-11-07 16:41:46.836305: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-11-07 16:41:46.836613: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter 

## Local training / Gcloud

In [14]:
%%bash
rm -rf model_trained_test
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=~/event-driven-ml/edml-trainer \
   --job-dir=./tmp \
   -- \
   --train-examples=500 \
   --output-dir=model_trained_test \
   --pattern="*" \
   --eval-steps=1

ERROR: (gcloud.ai-platform.local.train) /usr/bin/python: command not found


CalledProcessError: Command 'b'rm -rf model_trained_test\ngcloud ai-platform local train \\\n   --module-name=trainer.task \\\n   --package-path=~/event-driven-ml/edml-trainer \\\n   --job-dir=./tmp \\\n   -- \\\n   --train-examples=500 \\\n   --output-dir=model_trained_test \\\n   --pattern="*" \\\n   --eval-steps=1\n'' returned non-zero exit status 1